In [1]:
import os

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests
import json

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

In [2]:
directory_in = './file/8.kit_part'

list_file = [file for file in sorted(os.listdir(directory_in)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_in}/{file}')]
file = list_file[0]

file

'crawler_1.xlsx'

In [3]:
list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(f'{directory_in}/{file}',
                                                                       header=0,
                                                                       dtype=str).fillna('')

count_remain = len(df_input)

print(f'总数量：{count_remain}')
print()
df_input

总数量：884



,[Kit] JOIN_MPNTCPC,[Kit] Type,[Kit] Manufacturer,[Kit] Part Number,[Kit] OEM,[Kit] Picture,[Kit] Url,[Kit] Json_Src,[Kit] Info,[Kit] Json_Note 1,[Kit] Json_Note 2,[Kit] Choose,[Kit] Json_Price,[Kit] Type Code,[Kit] Part Code
0,MELLING;1007;5724;8304536,Timing Chain,MELLING,1007,12566651;12586964;12590920;12597307,,"https://www.rockauto.com/en/parts/MELLING,1007...","{""0"": ""https://www.rockauto.com/info/583/1007-...",https://www.rockauto.com/en/moreinfo.php?pk=83...,{},"{""0"": ""Category: Timing Chain""}",,"{""0"": ""$31.79""}",5724,8304536
1,MELLING;1025;5724;8921164,Timing Chain,MELLING,1025,135060P010;135060P011;1350631020;1350631030;13...,,"https://www.rockauto.com/en/parts/MELLING,1025...","{""0"": ""https://www.rockauto.com/info/583/1025-...",https://www.rockauto.com/en/moreinfo.php?pk=89...,{},"{""0"": ""Category: Timing Chain""}",,"{""0"": ""$59.79""}",5724,8921164
2,MELLING;1026;5724;8921168,Timing Chain,MELLING,1026,135070P010;1350731020,,"https://www.rockauto.com/en/parts/MELLING,1026...","{""0"": ""https://www.rockauto.com/info/583/1026-...",https://www.rockauto.com/en/moreinfo.php?pk=89...,{},"{""0"": ""Category: Timing Chain""}",,"{""0"": ""$12.03""}",5724,8921168
3,MELLING;1035;5724;10454488,Timing Chain,MELLING,1035,14401RNAA01,,"https://www.rockauto.com/en/parts/MELLING,1035...","{""0"": ""https://www.rockauto.com/info/583/1035-...",https://www.rockauto.com/en/moreinfo.php?pk=10...,{},"{""0"": ""Category: Timing Chain""}",,"{""0"": ""$43.89""}",5724,10454488
4,MELLING;1036;5724;8921172,Timing Chain,MELLING,1036,BR3Z6268B,,"https://www.rockauto.com/en/parts/MELLING,1036...","{""0"": ""https://www.rockauto.com/info/583/1036-...",https://www.rockauto.com/en/moreinfo.php?pk=89...,{},"{""0"": ""Category: Timing Chain""}",,"{""0"": ""$35.79""}",5724,8921172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
879,MELLING;WP10022;2208;14645716,Water Pump,MELLING,WP10022,,,"https://www.rockauto.com/en/parts/MELLING,WP10...",{},https://www.rockauto.com/en/moreinfo.php?pk=14...,{},"{""0"": ""Category: Water Pump""}",,"{""0"": ""Out of Stock""}",2208,14645716
880,MELLING;WP10091;2208;14645720,Water Pump,MELLING,WP10091,,,"https://www.rockauto.com/en/parts/MELLING,WP10...",{},https://www.rockauto.com/en/moreinfo.php?pk=14...,{},"{""0"": ""Category: Water Pump""}",,"{""0"": ""Out of Stock""}",2208,14645720
881,MELLING;WP10510;2208;14645712,Water Pump,MELLING,WP10510,,,"https://www.rockauto.com/en/parts/MELLING,WP10...",{},https://www.rockauto.com/en/moreinfo.php?pk=14...,{},"{""0"": ""Category: Water Pump (Only 1 Remaining)""}",,"{""0"": ""$19.57""}",2208,14645712
882,MELLING;WP10516;2208;14645724,Water Pump,MELLING,WP10516,,,"https://www.rockauto.com/en/parts/MELLING,WP10...",{},https://www.rockauto.com/en/moreinfo.php?pk=14...,{},"{""0"": ""Category: Water Pump""}",,"{""0"": ""Out of Stock""}",2208,14645724


In [4]:
crawler_status, (crawler_index, crawler_series) = False, next(df_input.iterrows())

crawler_status, crawler_index, crawler_series

(False,
 0,
 [Kit] JOIN_MPNTCPC                            MELLING;1007;5724;8304536
 [Kit] Type                                                 Timing Chain
 [Kit] Manufacturer                                              MELLING
 [Kit] Part Number                                                  1007
 [Kit] OEM                           12566651;12586964;12590920;12597307
 [Kit] Picture                                                          
 [Kit] Url             https://www.rockauto.com/en/parts/MELLING,1007...
 [Kit] Json_Src        {"0": "https://www.rockauto.com/info/583/1007-...
 [Kit] Info            https://www.rockauto.com/en/moreinfo.php?pk=83...
 [Kit] Json_Note 1                                                    {}
 [Kit] Json_Note 2                       {"0": "Category: Timing Chain"}
 [Kit] Choose                                                           
 [Kit] Json_Price                                        {"0": "$31.79"}
 [Kit] Type Code                       

In [5]:
import time

data = {'func': 'getbuyersguide',
        'payload': json.dumps({'partData': {'listing_data_essential': {'parttype': crawler_series['[Kit] Type Code'],
                                                                       'partkey': crawler_series['[Kit] Part Code']},
                                            'listing_data_supplemental': {'partnumber': crawler_series['[Kit] Part Number'],
                                                                          'catalogname': crawler_series['[Kit] Manufacturer']}}}),
        'api_json_request': '1'}

count_retry = 0
while True:
    count_retry += 1
    print(count_retry)
    try:
        url_check = crawler_series['[Kit] Url']

        resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                             data=data,
                             headers=get_header(),
                             proxies=get_proxy(),
                             timeout=(5, 15))

        if resp.status_code == 200:
            dict_ = resp.json()

            if 'redirect_to_url' not in dict_ and dict_['buyersguidepieces']['title'].startswith('Buyer\'s Guide :') and 'collected_javascript' in dict_:
                break
    except KeyboardInterrupt:
        break
    except:
        time.sleep(3)
        continue

dict_

1


{'buyersguidepieces': {'body': '<div class="buyersguide-nested"><div style="padding: .5em;"><table class="nobmp" style="width: 100%;"><tr><td class="altrow-a-0">BUICK</td><td class="altrow-a-0">ALLURE</td><td class="altrow-a-0">2005-2006</td></tr><tr><td class="altrow-a-1">BUICK</td><td class="altrow-a-1">LACROSSE</td><td class="altrow-a-1">2005</td></tr><tr><td class="altrow-a-0">BUICK</td><td class="altrow-a-0">RENDEZVOUS</td><td class="altrow-a-0">2004-2006</td></tr><tr><td class="altrow-a-1">CADILLAC</td><td class="altrow-a-1">CTS</td><td class="altrow-a-1">2004-2007</td></tr><tr><td class="altrow-a-0">CADILLAC</td><td class="altrow-a-0">SRX</td><td class="altrow-a-0">2004-2006</td></tr><tr><td class="altrow-a-1">CADILLAC</td><td class="altrow-a-1">STS</td><td class="altrow-a-1">2005-2007</td></tr><tr><td class="altrow-a-0">SAAB</td><td class="altrow-a-0">9-3</td><td class="altrow-a-0">2006-2007</td></tr><tr><td class="altrow-a-1">SUZUKI</td><td class="altrow-a-1">XL-7</td><td clas

In [6]:
soup = BeautifulSoup(dict_['buyersguidepieces']['body'], 'lxml')
html = etree.HTML(str(soup))

with open('./kit_vehicle.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

<html>
 <body>
  <div class="buyersguide-nested">
   <div style="padding: .5em;">
    <table class="nobmp" style="width: 100%;">
     <tr>
      <td class="altrow-a-0">
       BUICK
      </td>
      <td class="altrow-a-0">
       ALLURE
      </td>
      <td class="altrow-a-0">
       2005-2006
      </td>
     </tr>
     <tr>
      <td class="altrow-a-1">
       BUICK
      </td>
      <td class="altrow-a-1">
       LACROSSE
      </td>
      <td class="altrow-a-1">
       2005
      </td>
     </tr>
     <tr>
      <td class="altrow-a-0">
       BUICK
      </td>
      <td class="altrow-a-0">
       RENDEZVOUS
      </td>
      <td class="altrow-a-0">
       2004-2006
      </td>
     </tr>
     <tr>
      <td class="altrow-a-1">
       CADILLAC
      </td>
      <td class="altrow-a-1">
       CTS
      </td>
      <td class="altrow-a-1">
       2004-2007
      </td>
     </tr>
     <tr>
      <td class="altrow-a-0">
       CADILLAC
      </td>
      <td class="altrow-a-0">
       S

In [7]:
list_tr = html.xpath('//tr')

list_tr

[<Element tr at 0x11fb0bf40>,
 <Element tr at 0x10820c980>,
 <Element tr at 0x120b25e00>,
 <Element tr at 0x120b27200>,
 <Element tr at 0x120b24940>,
 <Element tr at 0x120b24180>,
 <Element tr at 0x120b25040>,
 <Element tr at 0x120b24a00>]

In [8]:
not list_tr and not (html.xpath('//p/text()') and html.xpath('//p/text()')[0].strip() == 'No applications found.')

False

In [9]:
list_dict_correct_temp = []
for i, tr in enumerate(list_tr):
    list_text = [text.strip() for text in tr.xpath('./td/text()')]

    if len(list_text) == 3:
        list_year = list_text[2].split('-')
        year_begin, year_end = int(list_year[0]), int(list_year[-1])
        list_dict_correct_temp_temp = [{'[Kit] JOIN_MPNTCPC': crawler_series['[Kit] JOIN_MPNTCPC'],
                                        'Vehicle No': i + 1,
                                        'Make': list_text[0],
                                        'Model': list_text[1],
                                        'Year': year} for year in range(year_begin, year_end + 1)]
    elif len(list_text) == 2:
        list_year = list_text[1].split('-')
        year_begin, year_end = int(list_year[0]), int(list_year[-1])
        list_dict_correct_temp_temp = [{'[Kit] JOIN_MPNTCPC': crawler_series['[Kit] JOIN_MPNTCPC'],
                                        'Vehicle No': i + 1,
                                        'Make': list_text[0],
                                        'Year': year} for year in range(year_begin, year_end + 1)]
    else:
        print('fuck')

    list_dict_correct_temp.extend(list_dict_correct_temp_temp)

list_dict_correct_temp

[{'[Kit] JOIN_MPNTCPC': 'MELLING;1007;5724;8304536',
  'Vehicle No': 1,
  'Make': 'BUICK',
  'Model': 'ALLURE',
  'Year': 2005},
 {'[Kit] JOIN_MPNTCPC': 'MELLING;1007;5724;8304536',
  'Vehicle No': 1,
  'Make': 'BUICK',
  'Model': 'ALLURE',
  'Year': 2006},
 {'[Kit] JOIN_MPNTCPC': 'MELLING;1007;5724;8304536',
  'Vehicle No': 2,
  'Make': 'BUICK',
  'Model': 'LACROSSE',
  'Year': 2005},
 {'[Kit] JOIN_MPNTCPC': 'MELLING;1007;5724;8304536',
  'Vehicle No': 3,
  'Make': 'BUICK',
  'Model': 'RENDEZVOUS',
  'Year': 2004},
 {'[Kit] JOIN_MPNTCPC': 'MELLING;1007;5724;8304536',
  'Vehicle No': 3,
  'Make': 'BUICK',
  'Model': 'RENDEZVOUS',
  'Year': 2005},
 {'[Kit] JOIN_MPNTCPC': 'MELLING;1007;5724;8304536',
  'Vehicle No': 3,
  'Make': 'BUICK',
  'Model': 'RENDEZVOUS',
  'Year': 2006},
 {'[Kit] JOIN_MPNTCPC': 'MELLING;1007;5724;8304536',
  'Vehicle No': 4,
  'Make': 'CADILLAC',
  'Model': 'CTS',
  'Year': 2004},
 {'[Kit] JOIN_MPNTCPC': 'MELLING;1007;5724;8304536',
  'Vehicle No': 4,
  'Make': '

In [10]:
list_dict_correct.extend(list_dict_correct_temp)

list_dict_correct

[{'[Kit] JOIN_MPNTCPC': 'MELLING;1007;5724;8304536',
  'Vehicle No': 1,
  'Make': 'BUICK',
  'Model': 'ALLURE',
  'Year': 2005},
 {'[Kit] JOIN_MPNTCPC': 'MELLING;1007;5724;8304536',
  'Vehicle No': 1,
  'Make': 'BUICK',
  'Model': 'ALLURE',
  'Year': 2006},
 {'[Kit] JOIN_MPNTCPC': 'MELLING;1007;5724;8304536',
  'Vehicle No': 2,
  'Make': 'BUICK',
  'Model': 'LACROSSE',
  'Year': 2005},
 {'[Kit] JOIN_MPNTCPC': 'MELLING;1007;5724;8304536',
  'Vehicle No': 3,
  'Make': 'BUICK',
  'Model': 'RENDEZVOUS',
  'Year': 2004},
 {'[Kit] JOIN_MPNTCPC': 'MELLING;1007;5724;8304536',
  'Vehicle No': 3,
  'Make': 'BUICK',
  'Model': 'RENDEZVOUS',
  'Year': 2005},
 {'[Kit] JOIN_MPNTCPC': 'MELLING;1007;5724;8304536',
  'Vehicle No': 3,
  'Make': 'BUICK',
  'Model': 'RENDEZVOUS',
  'Year': 2006},
 {'[Kit] JOIN_MPNTCPC': 'MELLING;1007;5724;8304536',
  'Vehicle No': 4,
  'Make': 'CADILLAC',
  'Model': 'CTS',
  'Year': 2004},
 {'[Kit] JOIN_MPNTCPC': 'MELLING;1007;5724;8304536',
  'Vehicle No': 4,
  'Make': '

In [11]:
crawler_status = True

crawler_status

True

In [12]:
crawler_series['Check_Url'] = url_check

list_series_error.append(crawler_series)

list_series_error

[[Kit] JOIN_MPNTCPC                            MELLING;1007;5724;8304536
 [Kit] Type                                                 Timing Chain
 [Kit] Manufacturer                                              MELLING
 [Kit] Part Number                                                  1007
 [Kit] OEM                           12566651;12586964;12590920;12597307
 [Kit] Picture                                                          
 [Kit] Url             https://www.rockauto.com/en/parts/MELLING,1007...
 [Kit] Json_Src        {"0": "https://www.rockauto.com/info/583/1007-...
 [Kit] Info            https://www.rockauto.com/en/moreinfo.php?pk=83...
 [Kit] Json_Note 1                                                    {}
 [Kit] Json_Note 2                       {"0": "Category: Timing Chain"}
 [Kit] Choose                                                           
 [Kit] Json_Price                                        {"0": "$31.79"}
 [Kit] Type Code                                   

In [13]:
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['[Kit] JOIN_MPNTCPC', 'Vehicle No', 'Year'],
                                                             ascending=[True, True, False],
                                                             ignore_index=True)
    df_correct.to_excel('./test_kit_vehicle.xlsx', index=False)

df_correct

,[Kit] JOIN_MPNTCPC,Vehicle No,Make,Model,Year
0,MELLING;1007;5724;8304536,1,BUICK,ALLURE,2006
1,MELLING;1007;5724;8304536,1,BUICK,ALLURE,2005
2,MELLING;1007;5724;8304536,2,BUICK,LACROSSE,2005
3,MELLING;1007;5724;8304536,3,BUICK,RENDEZVOUS,2006
4,MELLING;1007;5724;8304536,3,BUICK,RENDEZVOUS,2005
5,MELLING;1007;5724;8304536,3,BUICK,RENDEZVOUS,2004
6,MELLING;1007;5724;8304536,4,CADILLAC,CTS,2007
7,MELLING;1007;5724;8304536,4,CADILLAC,CTS,2006
8,MELLING;1007;5724;8304536,4,CADILLAC,CTS,2005
9,MELLING;1007;5724;8304536,4,CADILLAC,CTS,2004


In [14]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['[Kit] JOIN_MPNTCPC'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel('./test_kit_vehicle_error.xlsx', index=False)

df_error

,[Kit] JOIN_MPNTCPC,[Kit] Type,[Kit] Manufacturer,[Kit] Part Number,[Kit] OEM,[Kit] Picture,[Kit] Url,[Kit] Json_Src,[Kit] Info,[Kit] Json_Note 1,[Kit] Json_Note 2,[Kit] Choose,[Kit] Json_Price,[Kit] Type Code,[Kit] Part Code,Check_Url
0,MELLING;1007;5724;8304536,Timing Chain,MELLING,1007,12566651;12586964;12590920;12597307,,"https://www.rockauto.com/en/parts/MELLING,1007...","{""0"": ""https://www.rockauto.com/info/583/1007-...",https://www.rockauto.com/en/moreinfo.php?pk=83...,{},"{""0"": ""Category: Timing Chain""}",,"{""0"": ""$31.79""}",5724,8304536,"https://www.rockauto.com/en/parts/MELLING,1007..."
